In [2]:
import pandas as pd
import numpy as np
def alpha(f1,f2):
    y = 1.64*(f1-f2)-2.82
    return y

# c2d CLOUDS

In [45]:
c2d = pd.read_csv("../Data_and_Results/c2d_CORES_HREL.csv")
print(c2d.info())

c2d.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109119 entries, 0 to 109118
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        109119 non-null  object 
 1   c2d_id        109119 non-null  object 
 2   ra            109119 non-null  float64
 3   dec           109119 non-null  float64
 4   alpha         109119 non-null  float64
 5   object_type   109119 non-null  object 
 6   j_flux_c      77553 non-null   float64
 7   j_d_flux_c    70380 non-null   float64
 8   h_flux_c      77553 non-null   float64
 9   h_d_flux_c    73398 non-null   float64
 10  ks_flux_c     77553 non-null   float64
 11  ks_d_flux_c   69770 non-null   float64
 12  ir1_flux_c    97120 non-null   float64
 13  ir1_d_flux_c  97119 non-null   float64
 14  ir2_flux_c    91384 non-null   float64
 15  ir2_d_flux_c  91384 non-null   float64
 16  ir3_flux_c    97233 non-null   float64
 17  ir3_d_flux_c  97233 non-null   float64
 18  ir4_

,region,c2d_id,ra,dec,alpha,object_type,j_flux_c,j_d_flux_c,h_flux_c,h_d_flux_c,...,ir2_flux_c,ir2_d_flux_c,ir3_flux_c,ir3_d_flux_c,ir4_flux_c,ir4_d_flux_c,mp1_flux_c,mp1_d_flux_c,mp2_flux_c,mp2_d_flux_c
0,CG30-31,SSTc2d J080834.0-355507,122.141464,-35.918583,-0.04,Galc,NaN,NaN,NaN,NaN,...,0.178,0.0138,0.305,0.0337,0.392,0.0511,0.876,0.165,NaN,NaN
1,CB68,SSTc2d J165702.8-160628,254.261780,-16.107676,0.44,Galc,NaN,NaN,NaN,NaN,...,0.433,0.0251,0.756,0.0590,1.240,0.0698,2.570,0.306,NaN,NaN
2,DC2910-35,SSTc2d J105955.6-634102,164.981842,-63.684002,0.29,Galc,NaN,NaN,NaN,NaN,...,0.242,0.0150,0.357,0.0390,0.598,0.0391,1.180,0.233,NaN,NaN
3,L1524-4,SSTc2d J042959.5+242724,67.497749,24.456640,0.69,Galc,NaN,NaN,NaN,NaN,...,0.186,0.0207,0.200,0.0333,1.830,0.1500,2.280,0.301,84.300003,11.0
4,L100,SSTc2d J171550.0-210046,258.958344,-21.012896,0.56,Galc,NaN,NaN,NaN,NaN,...,0.257,0.0157,0.472,0.0447,0.841,0.0514,1.690,0.271,NaN,NaN


In [46]:
c2d_IRAC = c2d[["object_type","region","ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c"]]
c2d_IRAC = c2d_IRAC[~c2d_IRAC["ir1_d_flux_c"].isna() & ~c2d_IRAC["ir2_flux_c"].isna() & ~c2d_IRAC["ir3_flux_c"].isna() & ~c2d_IRAC["ir4_flux_c"].isna()]
print(c2d_IRAC.info())
c2d_IRAC.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79106 entries, 0 to 109118
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   object_type   79106 non-null  object 
 1   region        79106 non-null  object 
 2   ir1_flux_c    79106 non-null  float64
 3   ir1_d_flux_c  79106 non-null  float64
 4   ir2_flux_c    79106 non-null  float64
 5   ir2_d_flux_c  79106 non-null  float64
 6   ir3_flux_c    79106 non-null  float64
 7   ir3_d_flux_c  79106 non-null  float64
 8   ir4_flux_c    79106 non-null  float64
 9   ir4_d_flux_c  79106 non-null  float64
dtypes: float64(8), object(2)
memory usage: 6.6+ MB
None


,object_type,region,ir1_flux_c,ir1_d_flux_c,ir2_flux_c,ir2_d_flux_c,ir3_flux_c,ir3_d_flux_c,ir4_flux_c,ir4_d_flux_c
0,Galc,CG30-31,0.164,0.0099,0.178,0.0138,0.305,0.0337,0.392,0.0511
1,Galc,CB68,0.239,0.0136,0.433,0.0251,0.756,0.0590,1.240,0.0698
2,Galc,DC2910-35,0.135,0.0171,0.242,0.0150,0.357,0.0390,0.598,0.0391
3,Galc,L1524-4,0.156,0.0163,0.186,0.0207,0.200,0.0333,1.830,0.1500
4,Galc,L100,0.159,0.0104,0.257,0.0157,0.472,0.0447,0.841,0.0514


In [47]:
pd.unique(c2d_IRAC["object_type"])

array(['Galc', 'Galc_PAH-em', 'Galc_red', 'Galc_star+dust(IR1)',
       'Galc_star+dust(IR2)', 'Galc_star+dust(IR3)',
       'Galc_star+dust(IR4)', 'Galc_star+dust(MP1)',
       'Galc_star+dust(MP2)', 'star', 'star+dust(IR1)', 'star+dust(IR2)',
       'star+dust(IR3)', 'star+dust(IR4)', 'star+dust(MP1)', 'YSOc',
       'YSOc_PAH-em', 'YSOc_red', 'YSOc_star+dust(IR1)',
       'YSOc_star+dust(IR2)', 'YSOc_star+dust(IR3)',
       'YSOc_star+dust(IR4)', 'YSOc_star+dust(MP1)',
       'YSOc_star+dust(MP2)'], dtype=object)

In [48]:
c2d_YSO_1 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_star+dust(MP1)'],c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_star+dust(IR4)'])
c2d_YSO_2 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_star+dust(IR2)'],c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_star+dust(IR3)'])
c2d_YSO_3 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_star+dust(IR1)'],c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_red'])
c2d_YSO_4 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='YSOc_PAH-em'],c2d_IRAC[c2d_IRAC["object_type"]=='YSOc'])
c2d_YSO_12 = pd.merge_ordered(c2d_YSO_1,c2d_YSO_2)
c2d_YSO_34 = pd.merge_ordered(c2d_YSO_3,c2d_YSO_4)
c2d_YSO = pd.merge_ordered(c2d_YSO_12,c2d_YSO_34)

c2d_YSO["Target"] = 0


c2d_EG_1 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(MP1)'],c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(IR4)'])
c2d_EG_2 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(IR2)'],c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(IR3)'])
c2d_EG_3 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(IR1)'],c2d_IRAC[c2d_IRAC["object_type"]=='Galc_red'])
c2d_EG_4 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='Galc_PAH-em'],c2d_IRAC[c2d_IRAC["object_type"]=='Galc'])
c2d_EG_12 = pd.merge_ordered(c2d_EG_1,c2d_EG_2)
c2d_EG_34 = pd.merge_ordered(c2d_EG_3,c2d_EG_4)
c2d_EG = pd.merge_ordered(c2d_EG_12,c2d_EG_34)
c2d_EG["Target"] = 1


c2d_ST_1 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='star+dust(MP1)'],c2d_IRAC[c2d_IRAC["object_type"]=='star+dust(IR4)'])
c2d_ST_2 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='star+dust(IR2)'],c2d_IRAC[c2d_IRAC["object_type"]=='star+dust(IR3)'])
c2d_ST_3 = pd.merge_ordered(c2d_IRAC[c2d_IRAC["object_type"]=='Galc_star+dust(IR1)'],c2d_IRAC[c2d_IRAC["object_type"]=='star'])
c2d_ST_12 = pd.merge_ordered(c2d_ST_1,c2d_ST_2)
c2d_ST = pd.merge_ordered(c2d_ST_3,c2d_ST_12)
c2d_ST["Target"] = 2

c2d_IRAC_1 = pd.merge_ordered(c2d_YSO,c2d_EG)
c2d_IRAC_NEW = pd.merge_ordered(c2d_IRAC_1,c2d_ST)

c2d_IRAC_NEW.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79097 entries, 0 to 79096
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   object_type   79097 non-null  object 
 1   region        79097 non-null  object 
 2   ir1_flux_c    79097 non-null  float64
 3   ir1_d_flux_c  79097 non-null  float64
 4   ir2_flux_c    79097 non-null  float64
 5   ir2_d_flux_c  79097 non-null  float64
 6   ir3_flux_c    79097 non-null  float64
 7   ir3_d_flux_c  79097 non-null  float64
 8   ir4_flux_c    79097 non-null  float64
 9   ir4_d_flux_c  79097 non-null  float64
 10  Target        79097 non-null  int64  
dtypes: float64(8), int64(1), object(2)
memory usage: 7.2+ MB


In [49]:
print(f"YSO shape: {c2d_YSO.shape}")
print(f"EG shape: {c2d_EG.shape}")
print(f"Star shape: {c2d_ST.shape}")

YSO shape: (314, 11)
EG shape: (515, 11)
Star shape: (78268, 11)


In [28]:
# Calculate alphas and add to be consisitent with available bands ([4.5]-[8])
c2d_IRAC_NEW["alpha"] = alpha(c2d_IRAC_NEW["ir2_flux_c"],c2d_IRAC_NEW["ir4_flux_c"])

c2d_IRAC_NEW.head()

,object_type,region,ir1_flux_c,ir1_d_flux_c,ir2_flux_c,ir2_d_flux_c,ir3_flux_c,ir3_d_flux_c,ir4_flux_c,ir4_d_flux_c,Target,alpha
0,Galc,B18-1,0.134,0.0114,0.171,0.0109,0.211,0.0334,0.288,0.0359,1,-3.01188
1,Galc,B35A,0.165,0.0104,0.224,0.0136,0.278,0.0338,0.436,0.0427,1,-3.16768
2,Galc,B35A,0.248,0.0129,0.469,0.0265,0.622,0.0424,0.780,0.0543,1,-3.33004
3,Galc,B35A,0.372,0.1040,1.360,0.4050,3.230,0.4790,19.100,2.3300,1,-31.91360
4,Galc,B35A,0.871,0.0425,1.200,0.0586,1.310,0.0722,1.370,0.0718,1,-3.09880


In [29]:
np.save("../Data_and_Results/c2d_Inputs_CORES_Valid.npy",c2d_IRAC_NEW[["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c","alpha"]].values.astype(float))
np.save("../Data_and_Results/c2d_Targets_CORES_Valid.npy",c2d_IRAC_NEW[["Target"]].values.astype(int))

# Rapson Data

In [36]:
df_Rap = pd.read_csv("../Data_and_Results/Rapson_YSO_EG_ST.tsv",sep=';',comment='#')
df_Rap = df_Rap[df_Rap["8mag"]!='     ']
df_Rap = df_Rap[df_Rap["5.8mag"]!='     '] 
df_Rap = df_Rap[df_Rap["4.5mag"]!='     ']
df_Rap = df_Rap[df_Rap["3.6mag"]!='     ']
df_Rap.info()

# pd.unique(df_Rap["4.5mag"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8444 entries, 4 to 10448
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        8444 non-null   int64 
 1   RAJ2000   8444 non-null   object
 2   DEJ2000   8444 non-null   object
 3   3.6mag    8444 non-null   object
 4   e_3.6mag  8444 non-null   object
 5   4.5mag    8444 non-null   object
 6   e_4.5mag  8444 non-null   object
 7   5.8mag    8444 non-null   object
 8   e_5.8mag  8444 non-null   object
 9   8mag      8444 non-null   object
 10  e_8mag    8444 non-null   object
 11  24mag     8444 non-null   object
 12  e_24mag   8444 non-null   object
 13  Class     8444 non-null   object
dtypes: int64(1), object(13)
memory usage: 989.5+ KB


In [37]:
Rap_YSO_1 = pd.merge_ordered(df_Rap[df_Rap["Class"]=='0/I'],df_Rap[df_Rap["Class"]=='II'])
Rap_YSO = pd.merge_ordered(Rap_YSO_1,df_Rap[df_Rap["Class"]=='TD'])
Rap_YSO["Target"] = 0

Rap_EG_1 = pd.merge_ordered(df_Rap[df_Rap["Class"]=='AGN'],df_Rap[df_Rap["Class"]=='PAH'])
Rap_EG = pd.merge_ordered(Rap_EG_1,df_Rap[df_Rap["Class"]=='SHOCK'])
Rap_EG["Target"] = 1

Rap_ST = df_Rap[df_Rap["Class"]=="III/F"]
Rap_ST["Target"] = 2

Rap_NEW_1 = pd.merge_ordered(Rap_YSO,Rap_EG)
Rap_NEW = pd.merge_ordered(Rap_NEW_1, Rap_ST)

print(f"YSO shape: {Rap_YSO.shape}")
print(f"EG shape: {Rap_EG.shape}")
print(f"Stars shape: {Rap_ST.shape}")


Rap_NEW.info()

YSO shape: (553, 15)
EG shape: (373, 15)
Stars shape: (7518, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8444 entries, 0 to 8443
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        8444 non-null   int64 
 1   RAJ2000   8444 non-null   object
 2   DEJ2000   8444 non-null   object
 3   3.6mag    8444 non-null   object
 4   e_3.6mag  8444 non-null   object
 5   4.5mag    8444 non-null   object
 6   e_4.5mag  8444 non-null   object
 7   5.8mag    8444 non-null   object
 8   e_5.8mag  8444 non-null   object
 9   8mag      8444 non-null   object
 10  e_8mag    8444 non-null   object
 11  24mag     8444 non-null   object
 12  e_24mag   8444 non-null   object
 13  Class     8444 non-null   object
 14  Target    8444 non-null   int64 
dtypes: int64(2), object(13)
memory usage: 1.0+ MB


/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_1239/2374731623.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rap_ST["Target"] = 2


In [38]:
# Calculate alphas and add to be consisitent with available bands ([4.5]-[8])
Rap_NEW["alpha"] = alpha(Rap_NEW["4.5mag"].values.astype(float),Rap_NEW["8mag"].values.astype(float))

Rap_NEW.head()

,ID,RAJ2000,DEJ2000,3.6mag,e_3.6mag,4.5mag,e_4.5mag,5.8mag,e_5.8mag,8mag,e_8mag,24mag,e_24mag,Class,Target,alpha
0,34,06 43 10.32,+08 45 11.33,10.32,0.003,10.17,0.003,10.07,0.005,10.03,0.008,,,III/F,2,-2.5904
1,45,06 42 42.63,+08 45 20.25,11.62,0.003,11.60,0.004,11.55,0.016,11.56,0.035,,,III/F,2,-2.7544
2,54,06 43 02.63,+08 45 23.95,13.38,0.007,13.28,0.011,13.27,0.050,13.78,0.155,,,III/F,2,-3.6400
3,65,06 43 01.89,+08 45 32.18,12.52,0.005,12.52,0.006,12.40,0.030,12.41,0.053,,,III/F,2,-2.6396
4,84,06 42 53.88,+08 45 38.50,13.71,0.009,13.66,0.013,13.60,0.069,13.67,0.148,,,III/F,2,-2.8364


In [39]:
np.save("../Data_and_Results/Rap_Inputs_Test.npy",Rap_NEW[["3.6mag","e_3.6mag","4.5mag","e_4.5mag","5.8mag","e_5.8mag","8mag","e_8mag","alpha"]].values.astype(float))
np.save("../Data_and_Results/Rap_Targets_Test.npy",Rap_NEW[["Target"]].values.astype(int))

# Data from CM21


If we follow the values CM21 used for each class, we should be using:
* Training
    * 1472 YSOs (331 CI and 1141 CII)
    * 857 EG (231 Gals, 529 AGN, 27 PAHs 70 Shocks)
    * 1257 Stars
* Validation
    * 613 YSOs (82 CI and 531 CII)
    * 405 EG (104 Gals, 278 AGNs, 6 PAHs, 17 Shocks)
    * 4359 Stars

We will use the above input and compare to using 1000 of each class. There are a total of:
* 3073 YSOs
* 2034 EG
* 21 796 Stars


# SPICY YSO Classification

In [5]:
df_SPICY = pd.read_csv("../Data_and_Results/SPICY_YSO_Subclasses.csv",comment='#')

df_SPICY.head()

,SPICY,RAdeg DEdeg,GLON GLAT,p1,p2,p3,class,Si,PAH,alpha,...,ag.1,ag.2,ag.3,2MASS,UKIDSS,VIRAC,GaiaDR2,MIPS,WISEA,ZTFDR3
0,1,125.664811 -37.551349,255.975183 -0.253369,0.710000,,,ClassII,0,0,-1.51695368300395E+00,...,2,2,2,08223956-3733048,,,5541983493883598976,,J082239.55-373304.5,NaN
1,2,125.795513 -37.585000,256.062070 -0.187675,0.690000,,,ClassII,0,0,-6.43206342197073E-01,...,2,2,2,08231092-3735060,,,5541980848188474496,,J082310.90-373505.9,NaN
2,3,125.860244 -37.577463,256.085297 -0.141319,0.870000,,,FS,0,0,-2.14090416881411E-03,...,2,2,2,08232646-3734388,,,5541969475116705664,,J082326.45-373438.8,NaN
3,4,125.946863 -37.770809,256.282985 -0.196119,0.530000,,,ClassI,0,0,3.37295842881326E-01,...,4,2,4,,,,,,J082347.25-374615.0,NaN
4,5,126.003155 -37.126075,255.780917 0.210831,0.540000,,,FS,0,0,-2.78597489454348E-01,...,2,2,2,08240071-3707341,,,5542020537982099840,,J082400.75-370733.7,NaN


In [6]:
df_SPICY_IRAC = df_SPICY[[" SPICY","3.6mag","e_3.6mag","4.5mag","e_4.5mag","5.8mag","e_5.8mag","8.0mag","e_8.0mag","class    "]]
df_SPICY_IRAC = df_SPICY_IRAC[df_SPICY_IRAC["8.0mag"]!='      ']
df_SPICY_IRAC = df_SPICY_IRAC[df_SPICY_IRAC["5.8mag"]!='      ']

df_SPICY_IRAC.head()

,SPICY,3.6mag,e_3.6mag,4.5mag,e_4.5mag,5.8mag,e_5.8mag,8.0mag,e_8.0mag,class
0,1,13.158,0.063,12.800,0.104,12.470,0.153,12.009,0.083,ClassII
1,2,13.180,0.059,12.727,0.097,12.241,0.120,11.404,0.059,ClassII
2,3,11.703,0.052,11.197,0.061,10.674,0.059,9.576,0.034,FS
3,4,14.271,0.096,13.398,0.084,12.462,0.142,11.478,0.045,ClassI
4,5,13.198,0.065,12.393,0.084,11.726,0.077,10.848,0.048,FS


In [11]:
df_SPICY_CI = df_SPICY_IRAC[df_SPICY_IRAC["class    "]=="ClassI   "]
df_SPICY_CI["Target"] = 0

df_SPICY_CFS = df_SPICY_IRAC[df_SPICY_IRAC["class    "]=="FS       "]
df_SPICY_CFS["Target"] = 1

df_SPICY_CII = df_SPICY_IRAC[df_SPICY_IRAC["class    "]=="ClassII  "]
df_SPICY_CII["Target"] = 2

df_SPICY_CIII = df_SPICY_IRAC[df_SPICY_IRAC["class    "]=="ClassIII "]
df_SPICY_CIII["Target"] = 3

df_YSO_NEW_1 = pd.merge_ordered(df_SPICY_CI,df_SPICY_CII)
df_YSO_NEW_2 = pd.merge_ordered(df_SPICY_CFS,df_SPICY_CIII)
df_YSO_NEW = pd.merge_ordered(df_YSO_NEW_1,df_YSO_NEW_2)

df_YSO_NEW['alphas'] = alpha(df_YSO_NEW["4.5mag"].values.astype(float),df_YSO_NEW["8.0mag"].values.astype(float))

df_YSO_NEW.head()

/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_9120/443959090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SPICY_CI["Target"] = 0
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_9120/443959090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SPICY_CFS["Target"] = 1
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_9120/443959090.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,SPICY,3.6mag,e_3.6mag,4.5mag,e_4.5mag,5.8mag,e_5.8mag,8.0mag,e_8.0mag,class,Target,alphas
0,1,13.158,0.063,12.800,0.104,12.470,0.153,12.009,0.083,ClassII,2,-1.52276
1,2,13.180,0.059,12.727,0.097,12.241,0.120,11.404,0.059,ClassII,2,-0.65028
2,3,11.703,0.052,11.197,0.061,10.674,0.059,9.576,0.034,FS,1,-0.16156
3,4,14.271,0.096,13.398,0.084,12.462,0.142,11.478,0.045,ClassI,0,0.32880
4,5,13.198,0.065,12.393,0.084,11.726,0.077,10.848,0.048,FS,1,-0.28620


In [12]:
Inputs = df_YSO_NEW[["3.6mag","e_3.6mag","4.5mag","e_4.5mag","5.8mag","e_5.8mag","8.0mag","e_8.0mag","alphas"]].values.astype(float)
Targets = df_YSO_NEW[["Target"]].values.astype(float)


np.save("../Data_and_Results/Inputs_YSO_Train.npy",Inputs)
np.save("../Data_and_Results/Targets_YSO_Train.npy",Targets)

In [10]:
print(df_SPICY_CI.shape)
print(df_SPICY_CII.shape)
print(df_SPICY_CFS.shape)
print(df_SPICY_CIII.shape)

(15596, 11)
(59561, 11)
(23563, 11)
(5333, 11)
